In [1]:
import os

OUTPUT_DIR = './labse_v4_good'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [2]:
class CFG:
    apex=True
    print_freq=100
    num_workers=10
    model="sentence-transformers/LaBSE"
    scheduler='linear' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=10
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=256
    fc_dropout=0.1
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    extra=False

In [3]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
import ast
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true
# %env CUDA_LAUNCH_BLOCKING=1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.13.3
transformers.__version__: 4.30.1
env: TOKENIZERS_PARALLELISM=true


In [4]:
def get_logger(filename=os.path.join(OUTPUT_DIR, "train")):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [5]:
import re

def probs2str(results, texts): # (4501, 70, 3)
    predictions = []
    classes = np.argmax(results, axis=-1)
    input_ids_list = [tokenizer(text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)["input_ids"] for text in texts]

    for clas, input_ids in zip(classes, input_ids_list):
        entities = convert_ids_to_entities(clas, input_ids)

        entities = list(itertools.chain(*entities))
        prediction = tokenizer.decode(entities, skip_special_tokens=True)
        prediction = re.sub(" - ", "-", prediction)
        prediction = re.sub(" & ", "&", prediction)
        prediction = re.sub("##", "", prediction)
        predictions.append(prediction)
    
    return predictions

def get_score_f1(y_true, y_pred):
    tp, fp, fn = 0, 0, 0
    acc = 0
    # print(y_pred)
    for tr, pr in zip(y_true, y_pred):
        if tr == pr: acc += 1
        pr = frozenset([pr])
        tr = frozenset([tr])

        tp += len(pr & tr)
        fp += len(pr - tr)
        fn += len(tr - pr)

    if tp == 0:
        return 0.0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    
    print(f"Accuracy: {acc / len(y_true)}")
    score = 2 / (1 / precision + 1 / recall)

    return score

In [6]:
def convert_ids_to_entities(ids, tokens=None, o_id=0, b_id=1, i_id=2):
    entities = []
    entity = []
    is_entity_started = False
    for index, token in enumerate(ids):
        if not is_entity_started and token == b_id:
            is_entity_started = True
            entity.append(index)
        elif is_entity_started and token == i_id:
            entity.append(index)
    
        if (is_entity_started and token == o_id) or (index == (len(ids) - 1)):
            is_entity_started = False
            entities.append(entity)
            entity = []
                
    if tokens is not None:
        tokens = np.array(tokens)
        token_entities = []
        for entity in entities:
            entity = np.array(entity)
            if len(entity) > 0:
                token_entity = tokens[entity]
                token_entities.append(token_entity.tolist())
            
        return token_entities
            
    return entities

In [7]:
import re

def preprocess_text(text):
    text = text.lower()
    # text = re.sub("_", " ", text)
    # text = re.sub("\d{5,}", " ", text) # убираю длиньше чем есть в метках
    # text = re.sub("\d+[\.\,]?\d* *(см|гр|г|л|мл|кг|шт|мкм|м)", " ", text) # убираю единицы измерения
    # text = re.sub("\d+[\.\,]?\d*\s?%", " ", text) # процентное содержание в товарах
    # text = re.sub("\[.*\]", " ", text) # убираем скобки и содержимое
    # text = re.sub("<.+>", " ", text) # убираем скобки и содержимое
    # text = re.sub("{.+}", " ", text) # убираем скобки и содержимое
    # for el in ["\x07", "\t", "\n", "\x18", "\x1a", '"', ",", "/", ":", ";", '\\\\', '\|', '~', "\x7f", "\xa0", "°", "·", "є", "∙", "╣"]:
    #     text = re.sub(el, " ", text)

    return text.strip()

def get_good_pos(row):
    if row["good"] == "": return (0, 0)
    start = row["name"].lower().find(row["good"])
    if start == -1:
        end = -1
    else:
        end = start + len(row["good"])

    return (start, end)

In [8]:
df = pd.read_csv("train_supervised_dataset.csv")
df.good = df.good.replace("лента.", "лента")
df.good = df.good.replace("товара нет", np.nan)
df.good = df.good.replace("т,а,б,л,е,т,к,и", "таблетки")
df.good = df.good.replace('автокормушка", "автопоилка', "автокормушка,автопоилка")

df = df[["name", "good"]].fillna("")
df.name = df.name.apply(preprocess_text)
df["good_pos"] = df.apply(get_good_pos, axis=1)
df = df[df.good_pos != (-1,-1)].reset_index(drop=True)

df["strat"] = df.good_pos.apply(lambda x: 1 if x == (0, 0) else 0)
print(df.shape)
df.head()

(23412, 4)


,name,good,good_pos,strat
0,petmax бантик леопард с красн розой 2шт,бантик,"(7, 13)",0
1,87191 бусы для елки шарики_87191,бусы,"(6, 10)",0
2,футболка piazza italia wr011446881,футболка,"(0, 8)",0
3,7) yi572-03x-one заколка для волос для девочки,заколка,"(17, 24)",0
4,одежда (вес) 1500,одежда,"(0, 6)",0


In [9]:
Fold = StratifiedKFold(n_splits=CFG.n_fold)
for n, (train_index, val_index) in enumerate(Fold.split(df, df.strat.values)):
    df.loc[val_index, 'fold'] = int(n)
df['fold'] = df['fold'].astype(int)
display(df.groupby('fold').size())

fold
0    4683
1    4683
2    4682
3    4682
4    4682
dtype: int64

In [10]:
if CFG.extra:
    extra_data = pd.read_csv("unsup_pred_rubert.csv")
    extra_data = extra_data[["name", "good"]].fillna("")
    extra_data.name = extra_data.name.apply(preprocess_text)
    extra_data["good_pos"] = extra_data.apply(get_good_pos, axis=1)
    extra_data = extra_data[extra_data.good_pos != (-1,-1)].reset_index(drop=True)
    extra_data.head()

In [11]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "tokenizer"))
CFG.tokenizer = tokenizer

In [12]:
lengths = []
tk0 = tqdm(df.name.values, total=len(df))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)

if CFG.extra:
    tk1 = tqdm(extra_data.name.values, total=len(extra_data))
    for text in tk1:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)

CFG.max_len = max(lengths) + 2 # cls & sep
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/23412 [00:00<?, ?it/s]

max_len: 66


In [13]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


def create_label(cfg, text, annotation, pos):
    # "O" -> 0
    # "B" -> 1
    # "I" -> 2
    encoded = cfg.tokenizer(text,
                            add_special_tokens=True,
                            max_length=CFG.max_len,
                            padding="max_length",
                            return_offsets_mapping=True)
    offset_mapping = encoded['offset_mapping']
    ignore_idxes = np.where(np.array(encoded.sequence_ids()) != 0)[0]
    label = np.zeros(len(offset_mapping))
    label[ignore_idxes] = -1

    if pos == (0, 0):
        return torch.tensor(label, dtype=torch.long)
    for i in range(len(offset_mapping)):
        if offset_mapping[i] == (0, 0):
            continue
        elif offset_mapping[i][0] == pos[0]:
            label[i] = 1
        elif offset_mapping[i][1] <= pos[1] and offset_mapping[i][0] >= pos[0]:
            label[i] = 2

    return torch.tensor(label, dtype=torch.long)

class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.name = df['name'].values
        self.good = df['good'].values
        self.good_pos = df['good_pos'].values

    def __len__(self):
        return len(self.name)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.name[item])
        label = create_label(self.cfg, 
                             self.name[item], 
                             self.good[item], 
                             self.good_pos[item])
        return inputs, label

In [14]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 3)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [16]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        loss = criterion(y_preds.transpose(1, 2), labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))

    return losses.avg

In [17]:
def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds.transpose(1, 2), labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.softmax(-1).to('cpu').numpy()) ####
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [18]:
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)

    if CFG.extra:
        train_folds = pd.concat([train_folds, extra_data], axis=0)
    
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_texts = valid_folds['name'].values
    valid_labels = valid_folds['good'].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, os.path.join(OUTPUT_DIR, "config.pth"))
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    
    best_score = 0.0

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        # scoring
        preds = probs2str(predictions, valid_texts)
        score = get_score_f1(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        
        if best_score < score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': np.argmax(predictions, axis=-1)},
                        os.path.join(OUTPUT_DIR, f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth"))

    predictions = torch.load(os.path.join(OUTPUT_DIR, f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth"), 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[i for i in range(CFG.max_len)]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [19]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        predictions = []
        classes = oof_df[[i for i in range(CFG.max_len)]].values.tolist()
        input_ids_list = [CFG.tokenizer(text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)["input_ids"] for text in oof_df.name.values]
        for clas, input_ids in zip(classes, input_ids_list):
            entities = convert_ids_to_entities(clas, input_ids)
    
            entities = list(itertools.chain(*entities))
            # entities = tokenizer.convert_ids_to_tokens(entities)
            # entities = [entity for entity in entities if entity != "[PAD]"]
            # prediction = tokenizer.convert_tokens_to_string(entities)
            prediction = tokenizer.decode(entities, skip_special_tokens=True)
            prediction = re.sub(" - ", "-", prediction)
            prediction = re.sub(" & ", "&", prediction)
            prediction = re.sub("##", "", prediction)
            predictions.append(prediction)

        score = get_score_f1(oof_df.good.values, predictions)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(df, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(os.path.join(OUTPUT_DIR, "oof_df.pkl"))

========== fold: 0 training ==========


Epoch: [1][0/73] Elapsed 0m 0s (remain 1m 7s) Loss: 1.0244(1.0244) Grad: nan  LR: 0.00001997  
Epoch: [1][72/73] Elapsed 0m 19s (remain 0m 0s) Loss: 0.0505(0.2408) Grad: 6431.5391  LR: 0.00001800  
EVAL: [0/19] Elapsed 0m 0s (remain 0m 6s) Loss: 0.0545(0.0545) 
EVAL: [18/19] Elapsed 0m 3s (remain 0m 0s) Loss: 0.1391(0.0422) 


Epoch 1 - avg_train_loss: 0.2408  avg_val_loss: 0.0422  time: 23s
Epoch 1 - Score: 0.9216
Epoch 1 - Save Best Score: 0.9216 Model


Accuracy: 0.9216314328421952
Epoch: [2][0/73] Elapsed 0m 0s (remain 0m 30s) Loss: 0.0386(0.0386) Grad: nan  LR: 0.00001798  
Epoch: [2][72/73] Elapsed 0m 19s (remain 0m 0s) Loss: 0.0405(0.0420) Grad: 10838.8818  LR: 0.00001601  
EVAL: [0/19] Elapsed 0m 0s (remain 0m 7s) Loss: 0.0504(0.0504) 
EVAL: [18/19] Elapsed 0m 3s (remain 0m 0s) Loss: 0.1165(0.0347) 


Epoch 2 - avg_train_loss: 0.0420  avg_val_loss: 0.0347  time: 23s
Epoch 2 - Score: 0.9357
Epoch 2 - Save Best Score: 0.9357 Model


...

Accuracy: 0.9455360956856045


========== fold: 4 result ==========
Score: 0.9462
========== CV ==========


Accuracy: 0.9461768475010679


Score: 0.9455


Accuracy: 0.9454553220570647


In [20]:
# 0.9367
# 0.9427